In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import time

In [1]:
!pip show ortools

Name: ortools
Version: 7.8.7959
Summary: Google OR-Tools python libraries and modules
Home-page: https://developers.google.com/optimization/
Author: Google Inc
Author-email: lperron@google.com
License: Apache 2.0
Location: c:\users\gg098b\appdata\local\continuum\anaconda3\lib\site-packages
Requires: protobuf, six
Required-by: 


In [2]:
dfModel = pd.read_csv('model.csv')
dfDistMat1= pd.read_csv('DistMat1.csv')
dfDistMat2= pd.read_csv('DistMat2.csv')
matDist = dfDistMat1.pivot(index='idTF', columns='idTT', values='dist').values
matTime = dfDistMat2.pivot(index='idTF', columns='idTT', values='Time').values

In [3]:
dfDistMat2
dfModel

,Unnamed: 0,idT,capdem,type,tw1,tw2,dist,allowed
0,0,0,51000,tech,48000,105000,15,NaN
1,1,1,50000,tech,49000,105000,15,NaN
2,2,2,50000,tech,49000,105000,15,NaN
3,3,3,50000,tech,49000,105000,15,NaN
4,4,4,51000,tech,48000,105000,15,NaN
...,...,...,...,...,...,...,...,...
1207,1207,1207,7000,job,69000,96000,15,"[0, 4, 5, 6, 7, 9, 10, 13, 15, 16, 17, 18, 20,..."
1208,1208,1208,14700,job,69000,96000,15,"[0, 4, 5, 6, 7, 9, 10, 13, 15, 16, 17, 18, 20,..."
1209,1209,1209,7000,job,69000,96000,15,"[0, 4, 5, 6, 7, 9, 10, 13, 15, 16, 17, 18, 20,..."
1210,1210,1210,9300,job,48000,72000,15,"[1, 2, 3, 8, 11, 12, 14, 19, 22, 23, 24, 28, 3..."


In [4]:
matJobsTW  = dfModel[dfModel.type=='job'][['tw1','tw2']].values
matTechsTW = dfModel[dfModel.type=='tech'][['tw1','tw2']].values
matJobsTW=matJobsTW.astype(int)
matTechsTW=matTechsTW.astype(int)

matJobDemand=dfModel[dfModel.type=='job']['capdem'].values
matJobDemand = matJobDemand.astype(int)

#Depot list (Same as Tech)
matDepot = dfModel[dfModel.type=='tech']['idT'].values.tolist()
matStart = dfModel[dfModel['type']=='tech']['idT'].values.tolist()
matEnd = dfModel[dfModel['type']=='tech']['idT'].values.tolist()
matLocAllowedTec = dfModel[dfModel['type']=='job']['allowed'].values.tolist() 
matTechCapapcity =  dfModel[dfModel['type']=='tech']['capdem'].values.tolist()

In [5]:
NO_OF_TECH_LOCS=len(matDepot)


In [6]:
def create_data_model():
    data = {}
    data['distance_matrix']=matDist.astype(int)
    data['time_matrix']=matTime.astype(int)
    data['num_techs'] = len(matTechsTW)
    data['starts'] = matStart
    data['ends']   = matEnd
    data['depot']  = matDepot
    data['RateHr'] = 1
    data['RateKm'] = 2
    data['demands'] = matJobDemand
    data["tech_capacities"] =matTechCapapcity
    data["LocAllowedTec"]=matLocAllowedTec
    return data

In [7]:
data =  create_data_model()

In [8]:
manager=pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_techs'], data['starts'],
                                       data['ends'])
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [9]:
## Add disjunctions

LST_PRIORITY_JOBS_NodeId=[]
PENALTY_DISJUNCTION=9999999
penalty = PENALTY_DISJUNCTION
NTotal=len(data['distance_matrix'])
print(NTotal)
for node in range(1, NTotal):
     if node in LST_PRIORITY_JOBS_NodeId:
        pass
     else:
        index=manager.NodeToIndex(node)
        #print(node,index)
        if index>=0:
             routing.AddDisjunction([index], penalty)  

1212


In [10]:
## Cost call back

def cost_callback(from_index, to_index):
    from_node = manager.IndexToNode(int(from_index))
    to_node = manager.IndexToNode(int(to_index))
    vehicle=routing.VehicleIndex(to_index)
    cost_d=int(data['distance_matrix'][from_node][to_node]*COST_DIST_M)
    cost_t=int(data['time_matrix'][from_node][to_node]*COST_TIME_M)
    cost=int(cost_d+cost_t)#+cost_e
    return cost
cost_callback_index = routing.RegisterTransitCallback(cost_callback)
cost = 'Cost'
routing.AddDimension(
    cost_callback_index,
    0,  # allow waiting time
    10000000,  # maximum cost per technician
    True,  # Force start cumul to zero.
    cost)
cost_dimension = routing.GetDimensionOrDie(cost)

In [11]:
# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(cost_callback_index)

In [12]:
CONSTRUCTION_OPT ='AUTOMATIC'
SOLVER_OPT='AUTOMATIC'
SEARCH_TIME=60

In [ ]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
print('starting solver ')

   # Setting first solution heuristic.
if CONSTRUCTION_OPT =='AUTOMATIC':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
elif CONSTRUCTION_OPT =='PATH_CHEAPEST_ARC':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
elif CONSTRUCTION_OPT =='PATH_MOST_CONSTRAINED_ARC':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC)
elif CONSTRUCTION_OPT =='SAVINGS':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.SAVINGS)
elif CONSTRUCTION_OPT =='SWEEP':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.SWEEP)
elif CONSTRUCTION_OPT =='CHRISTOFIDES':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES)
elif CONSTRUCTION_OPT =='ALL_UNPERFORMED':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.ALL_UNPERFORMED)
elif CONSTRUCTION_OPT =='BEST_INSERTION':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.BEST_INSERTION)
elif CONSTRUCTION_OPT =='PARALLEL_CHEAPEST_INSERTION':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
elif CONSTRUCTION_OPT =='LOCAL_CHEAPEST_INSERTION':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION)
elif CONSTRUCTION_OPT =='GLOBAL_CHEAPEST_ARC':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC)
elif CONSTRUCTION_OPT =='FIRST_UNBOUND_MIN_VALUE':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.FIRST_UNBOUND_MIN_VALUE)
elif CONSTRUCTION_OPT =='LOCAL_CHEAPEST_ARC':
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC)
else:
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

if SOLVER_OPT=='GUIDED_LOCAL_SEARCH':
    search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

elif SOLVER_OPT=='AUTOMATIC':
    search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)

elif SOLVER_OPT=='GREEDY_DESCENT':
    search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT)

elif SOLVER_OPT=='SIMULATED_ANNEALING':
    search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)

elif SOLVER_OPT=='TABU_SEARCH':
    search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH)

elif SOLVER_OPT=='OBJECTIVE_TABU_SEARCH':
    search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.OBJECTIVE_TABU_SEARCH)

search_parameters.time_limit.seconds = SEARCH_TIME
search_parameters.log_search = True

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

starting solver 
